# This is  a research notebook to try to find correlation between hyperwave and volume 

In [ ]:
# !pip install --upgrade pip

In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='davzucky', api_key='aZw7LRJOSDcPJyIk2G0U')

## This is the place where you setup the symbol and phases

In [3]:
data_3DSystemsCorp ={
    'name':'3D SYSTEMS CORP',
    'symbol':'DDD',
    'price_range_size':5,
    'phases':{
        "phase1":(datetime(2000, 1, 1), datetime(2012, 5, 14)),
        "phase2":(datetime(2012, 5, 14), datetime(2013, 3, 18)),
        "phase3":(datetime(2013, 3, 18), datetime(2013, 10, 7)),
        "phase4":(datetime(2013, 10, 7), datetime(2014, 1, 14)),
        "phase5":(datetime(2014, 1, 14), datetime(2014, 5, 5)),
        "phase6":(datetime(2014, 5, 5), datetime(2014, 7, 7)),
        "phase7":(datetime(2014, 7, 7) , datetime(2015, 7, 6))
    }
}


data_Valeant = {
    'name':'Baush health Company. Ex Valeant',
    'symbol':'VRX',
    'price_range_size':10,
    'phases':{
        "phase1":(datetime(2008, 3, 10), datetime(2010, 6, 21)),
        "phase2":(datetime(2010, 6, 21), datetime(2012, 7, 9)),
        "phase3":(datetime(2012, 7, 9), datetime(2014, 10, 27)),
        "phase4":(datetime(2014, 10, 27), datetime(2015, 8, 10)),
        "phase5":(datetime(2015, 8, 10), datetime(2015, 11, 9)),
        "phase6":(datetime(2015, 11, 9), datetime(2015, 12, 28)),
        "phase7":(datetime(2015, 12, 28), datetime(2016, 10, 31))
    }
}


In [4]:
company_setup = data_Valeant
name = company_setup ['name'] 
symbol = company_setup ['symbol']
price_range_size = company_setup ['price_range_size']
phases = company_setup['phases']

# constant used for other computation
root_date = datetime(1800, 1, 1)

## Fetch the data

In [18]:
def get_raw_historical_data(symbol, timeframe='daily'):
    url_symbol = "https://www.investopedia.com/markets/api/partial/historical/?Symbol={}&Type=Historical+Prices&Timeframe={}&StartDate=Jan+01%2C+1900".format(symbol, timeframe)
    print("Url to fetch the data {}".format(url_symbol))
    print("fetching data for {} on {}...".format(symbol, timeframe))
    df_list = pd.read_html(url_symbol, header=0, parse_dates=True)
    df_price = df_list[0].dropna()
    return df_price

df_daily_price_raw = get_raw_historical_data(symbol, 'Daily')
df_weekly_price_raw = get_raw_historical_data(symbol, 'Weekly')

Url to fetch the data https://www.investopedia.com/markets/api/partial/historical/?Symbol=VRX&Type=Historical+Prices&Timeframe=Daily&StartDate=Jan+01%2C+1900
fetching data for VRX on Daily...


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Url to fetch the data https://www.investopedia.com/markets/api/partial/historical/?Symbol=VRX&Type=Historical+Prices&Timeframe=Weekly&StartDate=Jan+01%2C+1900
fetching data for VRX on Weekly...


In [20]:
def clean_raw_market_data(df):
    df.loc[:,('Date')] = pd.to_datetime(df.loc[:,('Date')])
    df = df.rename(columns={'Adj. Close':'Close'})
    df['IsPriceClosingUp'] = df.Close > df.Close.shift()
    return df

df_daily_price = clean_raw_market_data(df_daily_price_raw)
df_weekly_price = clean_raw_market_data(df_weekly_price_raw)

In [33]:
df_daily_price["IsPriceClosingUp"]


0       False
1        True
2       False
3        True
4       False
5       False
6       False
7        True
8       False
9        True
10       True
11      False
12       True
13      False
14       True
15       True
16      False
17      False
18      False
19       True
20      False
21       True
22      False
23      False
24      False
25      False
26       True
27      False
28      False
29       True
        ...  
7211     True
7212    False
7213    False
7214    False
7215    False
7216    False
7217    False
7218    False
7219    False
7220    False
7221    False
7222     True
7223     True
7224    False
7225    False
7226     True
7227    False
7228    False
7229    False
7230     True
7231    False
7232    False
7233     True
7234    False
7235    False
7236     True
7237    False
7238    False
7239    False
7240    False
Name: IsPriceClosingUp, Length: 7191, dtype: bool

In [23]:
def get_price_range(row, range_size=1, column_name = 'Close'):
    return int(row[column_name]/range_size) * range_size

In [30]:
df_phases = { k:df_daily_price.loc[ \
                ((df_daily_price['Date'] >= v[0]) & \
                (df_daily_price['Date'] < v[1]))] for k,v in phases.items()}

def get_df_with_price_range(df):
    df['price_range'] = df_daily_price.apply (lambda row: get_price_range (row, price_range_size),axis=1)
    return df 
    
df_phases_with_price_range = { k:get_df_with_price_range(df) for k,df in df_phases.items()}
    
# TODO - Need to update here and split Price up and Down before doing that
df_phases_prices = {k:df.groupby(['price_range']).sum() for k,df in df_phases_with_price_range.items()}


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [35]:
df_phases_with_price_range["phase1"]["IsPriceClosingUp"]

2031    False
2032     True
2033     True
2034     True
2035    False
2036    False
2037    False
2038    False
2039    False
2040    False
2041     True
2042     True
2043    False
2044    False
2045     True
2046    False
2047    False
2048    False
2049     True
2050    False
2051    False
2052     True
2053     True
2054     True
2055     True
2056     True
2057     True
2058    False
2059    False
2060    False
        ...  
2576     True
2577     True
2578    False
2579     True
2580    False
2581     True
2582     True
2583     True
2584    False
2585     True
2586     True
2587    False
2588    False
2589    False
2590     True
2591    False
2592    False
2593     True
2594     True
2595    False
2596    False
2597    False
2598    False
2599    False
2600    False
2601     True
2602     True
2603    False
2604     True
2605    False
Name: IsPriceClosingUp, Length: 575, dtype: bool

In [9]:
df_phases_price_ranges = [df[['Volume']].rename(columns={'Volume':k}) for k,df in df_phases_prices.items()]

In [10]:
df_price_summary = pd.concat(df_phases_price_ranges, axis=1)

In [11]:
data = [ go.Bar(x=df_price_summary[col_name],
            y=df_price_summary.index,
            name=col_name,
            orientation='h')
    for col_name in df_price_summary.columns.values]

In [12]:
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Volume Per price Range {}'.format(name))

In [13]:
df_weekly_hyperwave = df_weekly_price.loc[ \
                ((df_weekly_price['Date'] >= phases['phase1'][0]) & \
                (df_weekly_price['Date'] < phases['phase7'][1]))]

trace = go.Candlestick(x=df_weekly_hyperwave.Date,
                       open=df_weekly_hyperwave.Open,
                       high=df_weekly_hyperwave.High,
                       low=df_weekly_hyperwave.Low,
                       close=df_weekly_hyperwave.Close)
layout = go.Layout(
    xaxis = dict(
        rangeslider = dict(
            visible = False
        )
    )
)

data = [trace]

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='OHLC without Rangeslider')


In [98]:
result_regression = {
    'a':0,
    'b':0,
    'eps':0,
    'point1':(0,0),
    'point2':(0,0)
}

def get_line_for_points(x1, y1, x2, y2):
    a = (y2 - y1) / (x2 - x1) 
    b = y1 - (a * x1)
    return (a, b)



def regresse_hyperwave_phase(x_array, y_array):
    lines = []
    for outer in np.arange(0, len(x_array)-1, 1):
        for inner in np.arange(outer+1, len(x_array), 1):
            print("outer: {}, inner: {}".format(outer, inner))
            lines.append( get_line_for_points(
                    x_array[outer],
                    y_array[outer],
                    x_array[inner],
                    y_array[inner]
                ))
    return lines

x_array = df_weekly_price.loc[:10,'weeks'].values
y_array = df_weekly_price.loc[:10,'Close'].values

lines = regresse_hyperwave_phase(x_array, y_array)
print(lines)

outer: 0, inner: 1
outer: 0, inner: 2
outer: 0, inner: 3
outer: 0, inner: 4
outer: 0, inner: 5
outer: 0, inner: 6
outer: 0, inner: 7
outer: 0, inner: 8
outer: 0, inner: 9
outer: 0, inner: 10
outer: 1, inner: 2
outer: 1, inner: 3
outer: 1, inner: 4
outer: 1, inner: 5
outer: 1, inner: 6
outer: 1, inner: 7
outer: 1, inner: 8
outer: 1, inner: 9
outer: 1, inner: 10
outer: 2, inner: 3
outer: 2, inner: 4
outer: 2, inner: 5
outer: 2, inner: 6
outer: 2, inner: 7
outer: 2, inner: 8
outer: 2, inner: 9
outer: 2, inner: 10
outer: 3, inner: 4
outer: 3, inner: 5
outer: 3, inner: 6
outer: 3, inner: 7
outer: 3, inner: 8
outer: 3, inner: 9
outer: 3, inner: 10
outer: 4, inner: 5
outer: 4, inner: 6
outer: 4, inner: 7
outer: 4, inner: 8
outer: 4, inner: 9
outer: 4, inner: 10
outer: 5, inner: 6
outer: 5, inner: 7
outer: 5, inner: 8
outer: 5, inner: 9
outer: 5, inner: 10
outer: 6, inner: 7
outer: 6, inner: 8
outer: 6, inner: 9
outer: 6, inner: 10
outer: 7, inner: 8
outer: 7, inner: 9
outer: 7, inner: 10
oute

In [77]:
def get_weeks_since(row, start_date = root_date):
    return int(((row['Date'] - root_date) / pd.offsets.Day(1)) / 7)

df_weekly_price['weeks'] = df_weekly_price.apply (lambda row: get_weeks_since (row),axis=1)
    


In [97]:
print(df_weekly_price.loc[:10,'weeks'].values)
print(df_weekly_price.shape)

[11402 11401 11400 11399 11398 11397 11396 11395 11394 11393 11392]
(1489, 8)


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])